#### Load libraries

In [53]:
from numpy import array
import numpy as np
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.models import Sequential
from keras.layers import LSTM, SpatialDropout1D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
import pandas as pd
import time

In [2]:
from bs4 import BeautifulSoup
import re
import string
import nltk
from nltk.tokenize import WordPunctTokenizer

#### Load data : TRAIN & TEST

In [40]:
train = pd.read_csv("train_E6oV3lV.csv", encoding='ISO-8859-1')
test = pd.read_csv("test_tweets_anuFYb8.csv", encoding='ISO-8859-1')

#### Function to clean Data

In [4]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    tok = WordPunctTokenizer()

    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

#### Clean Train dataset

- Delete puntuations
- Delete letters which can create noise

In [5]:
training = train.tweet
dftrain = []
lentrain = []
for t in training:
    dftrain.append(tweet_cleaner(t))

Transform column labe to array for our model

In [6]:
label = array(train.label)

## Create Word Embedding

In [7]:
#train_words = set(text_to_word_sequence(dftrain))

In [8]:
vocab_size_train = 50
encoded_train = [one_hot(d, vocab_size_train) for d in dftrain]

In [9]:
max_length = 150 # Max number word in a sentence
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')

## Define the model

In [ ]:
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = padded_train.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

#### Fit the model

In [ ]:
Y = pd.get_dummies(train['label'])

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(padded_train,Y, test_size = 0.20, random_state = 36)

model.fit(X_train, Y_train, batch_size =batch_size, nb_epoch = 15,  verbose = 5)

In [12]:
#Evaluate the model

score,acc = model.evaluate(X_train, Y_train, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

Accuracy: 93.398411


## Predict Test dataset

In [13]:
# Clean Test dataset
testing = test.tweet
dftest = []
for t in testing:
    dftest.append(tweet_cleaner(t))

In [14]:
#Word Embedding
vocab_size_train = 50
encoded_test = [one_hot(d, vocab_size_train) for d in dftest]
max_length = 150 # Max number word in a sentence
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

In [24]:
#Predict Test data
classes = model.predict(padded_test, verbose=0)

In [25]:
classes

array([[0.14071979],
       [0.1947848 ],
       [0.02069573],
       ...,
       [0.14845377],
       [0.13597016],
       [0.02211074]], dtype=float32)

In [27]:
result = pd.DataFrame(classes)
frames = [test, result]
submission = pd.concat(frames, axis=1, join_axes=[test.index])
submission
submission = submission[['id',0]]
submission.describe()
submission.columns = ['id', 'label']
submission = submission[['id', 'label']]
submission.to_csv('sub15.csv')